In [ ]:
#different versio of numpy required for top2vec
!pip uninstall numpy
!pip install numpy==1.19.5

In [ ]:
#install the packages
# !pip install sentence-transformers
# !pip install nltk
# !pip install torch
# !pip install tensorflow
# !pip install tensorboard
!pip install top2vec
!pip install top2vec[sentence_encoders]
# !pip install top2vec[sentence_transformers]
# !pip install top2vec[indexing]

In [ ]:
#import required packages
import random
from tqdm import tqdm
import numpy as np
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
!git clone https://github.com/akashjorss/sentence_bert
!unzip sentence_bert/sentences.zip
import pickle
with open('sentences.bin', 'rb') as f:
  sentences = pickle.load(f)

In [ ]:
# import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector
#load tensorboards with magics
%tensorflow_version 2.x
%load_ext tensorboard

In [ ]:
documents = random.choices(sentences,k = 100000)
documents = [d.lower() for d in documents]

In [ ]:
from top2vec import Top2Vec
model = Top2Vec(documents, embedding_model='universal-sentence-encoder', workers=4)

In [ ]:
model.hierarchical_topic_reduction(100)
# model.get_topic_hierarchy()
# topic_sizes, topic_nums = model.get_topic_sizes()
print("Original Number of topics: ", model.get_num_topics())
print("Reduced number of topics: ", model.get_num_topics(reduced=True))

In [ ]:
#word similarity
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["swimming"], num_topics=3, reduced=False) #swimming, cleanliness #keywords_neg=stop,
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)

In [ ]:
#reduce the number of topics and do hierarchical clustering
# topic_words, word_scores, topic_nums = model.get_topics(reduced=True)
# topic_nums = random.choices(topic_nums, k=20)
# for topic in topic_nums:
#     model.generate_topic_wordcloud(topic)

In [ ]:
#get document vectors and plot using umap and tensorboard
#dim reduction using umap
import umap
umap_embeddings = umap.UMAP(n_neighbors=15,
                            n_components=3, 
                            metric='cosine').fit_transform(model.document_vectors)

In [ ]:
#visualize high dim data in embedding projector
# Set up a logs directory, so Tensorboard knows where to look for files
import numpy as np
import tensorflow as tf
log_dir='./logs/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save the weights we want to analyse as a variable. 
umap_weights = tf.Variable(random.choices(umap_embeddings, k=10000))
# Create a checkpoint from embedding, the filename and key are
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=umap_weights)
checkpoint.save(os.path.join(log_dir, "umap_embedding.ckpt"))

# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
# embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
!ls

In [ ]:
%tensorboard --logdir ./logs/

In [ ]:
#visualize word embeddings: get rid of stopwords, sentiment words, and take more sentences

In [ ]:
#function to process sentence to remove stopwords and sentiment words
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def process_sentence(s):
  s = s.lower()

  stop = stopwords.words('english')
  sentiment_tags = ['ADJ', 'ADV', 'RB', 'VBZ']


  words = word_tokenize(s)
  #remove stop words
  words = [w for w in words if w not in stop]
  #remove sentiment words
  pos_tags = nltk.pos_tag(words)
  words = [t[0] for t in pos_tags if t[1] not in sentiment_tags]
  s = " ".join(words)
  return s

In [ ]:
# random.choices(documents, k=10)
#sentence similarity
# s = "oh and it is an additional dollars per night to park your car there!"
# s = process_sentence(s)
# #process s to remove stopwords and sentiment words

# topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=s.split(), num_topics=3, reduced=False) #swimming, cleanliness #keywords_neg=stop,
# for topic in topic_nums:
#     model.generate_topic_wordcloud(topic)